In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from utils import request_wapi,get_forecast,create_df,send_message,get_date

In [212]:
query = 'La estrella'
api_key = API_KEY_WAPI

url = 'http://api.weatherapi.com/v1/forecast.json?key={}&q={}&days=1&aqi=no&alerts=no'.format(api_key,query)

In [213]:
response = requests.get(url).json()

In [214]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [215]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [216]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [217]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-20'

In [218]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [219]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Clear'

In [220]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

12.9

In [221]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [222]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

## Dataframe

In [223]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    rain_prov = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,temp,rain,rain_prov

In [224]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'):

    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<?, ?it/s]


In [225]:
datos

[('2023-06-20', '00', 'Clear', 12.9, 0, 0),
 ('2023-06-20', '01', 'Partly cloudy', 12.6, 0, 0),
 ('2023-06-20', '02', 'Partly cloudy', 12.4, 0, 0),
 ('2023-06-20', '03', 'Partly cloudy', 12.1, 0, 0),
 ('2023-06-20', '04', 'Clear', 12.0, 0, 0),
 ('2023-06-20', '05', 'Clear', 11.8, 0, 0),
 ('2023-06-20', '06', 'Sunny', 11.7, 0, 0),
 ('2023-06-20', '07', 'Sunny', 16.8, 0, 0),
 ('2023-06-20', '08', 'Sunny', 20.4, 0, 0),
 ('2023-06-20', '09', 'Sunny', 22.9, 0, 0),
 ('2023-06-20', '10', 'Sunny', 25.0, 0, 0),
 ('2023-06-20', '11', 'Sunny', 29.5, 0, 0),
 ('2023-06-20', '12', 'Sunny', 30.7, 0, 0),
 ('2023-06-20', '13', 'Sunny', 31.3, 0, 0),
 ('2023-06-20', '14', 'Sunny', 31.4, 0, 0),
 ('2023-06-20', '15', 'Sunny', 31.1, 0, 0),
 ('2023-06-20', '16', 'Sunny', 29.8, 0, 0),
 ('2023-06-20', '17', 'Sunny', 26.4, 0, 0),
 ('2023-06-20', '18', 'Sunny', 20.6, 0, 0),
 ('2023-06-20', '19', 'Clear', 18.3, 0, 0),
 ('2023-06-20', '20', 'Clear', 15.2, 0, 0),
 ('2023-06-20', '21', 'Clear', 14.5, 0, 0),
 ('2023-

In [226]:
col = ['Fecha','Hora','Condicion','Temp','Rain','Rain_prov']
df = pd.DataFrame(datos,columns=col)

In [227]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-20,00,Clear,12.9,0,0
1,2023-06-20,01,Partly cloudy,12.6,0,0
2,2023-06-20,02,Partly cloudy,12.4,0,0
3,2023-06-20,03,Partly cloudy,12.1,0,0
4,2023-06-20,04,Clear,12.0,0,0
5,2023-06-20,05,Clear,11.8,0,0
6,2023-06-20,06,Sunny,11.7,0,0
7,2023-06-20,07,Sunny,16.8,0,0
8,2023-06-20,08,Sunny,20.4,0,0
9,2023-06-20,09,Sunny,22.9,0,0


In [228]:
df_rain = df[(df['Rain'] == 1)]
df_redu = df_rain[['Hora','Temp','Condicion']]
df_redu.set_index('Hora',inplace = True)

In [229]:
from datetime import datetime

datetime.now().time().hour

17

In [230]:


df_temp = df[['Hora','Temp','Condicion']]


In [231]:
df_temp = df_temp[(df_temp['Hora'] == str(datetime.now().time().hour))]

In [232]:
df_temp.set_index('Hora',inplace = True)

In [233]:
df_temp

,Temp,Condicion
Hora,,
17,26.4,Sunny


In [234]:

if df_redu.empty:
    print('no hay plan arrunchis, no va a llover :C')
else:
    print('wuju')

no hay plan arrunchis, no va a llover :C


In [235]:
df_redu

,Temp,Condicion
Hora,,


In [236]:
PHONE_NUMBER

'+14027874717'

## Mensaje SMS desde twilio

In [237]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


In [238]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

In [239]:
'Hoy va a llover, abrígate bien :) \n\n\n ' + str(df_redu)
k = '+573154472305'
j = '+573185953816'

In [240]:


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure

client = Client(account_sid, auth_token)
if df_redu.empty:
    template = '\n Hoy no hay arrunchis, no va a llover :c \n\n\n ' + str(df_temp) 
else:
    template = '\n Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

message = client.messages \
    .create(
        
        body=template,
        from_=PHONE_NUMBER,
        to=k
     )

print('Mensaje enviado: ',message.sid, template)



Mensaje enviado:  SM1e82b758b02d420102e20ada9356358b 
 Hoy no hay arrunchis, no va a llover :c 


       Temp Condicion
Hora                
17    26.4     Sunny


In [154]:
'Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

'Hoy va a llover en Medellin, abrígate bien :) \n\n\n    Hora  Temp      Condicion\n0    00  12.4          Clear\n1    01  12.1          Clear\n2    02  12.0  Partly cloudy\n3    03  11.8  Partly cloudy\n4    04  11.7  Partly cloudy\n5    05  11.6  Partly cloudy\n6    06  11.5  Partly cloudy\n7    07  14.9          Sunny\n8    08  18.8          Sunny\n9    09  21.2          Sunny\n10   10  23.1          Sunny\n11   11  25.6          Sunny\n12   12  28.4          Sunny\n13   13  29.0          Sunny\n14   14  28.9          Sunny\n15   15  28.3          Sunny\n16   16  26.1          Sunny\n17   17  24.4          Sunny\n18   18  20.2          Sunny\n19   19  17.1          Clear\n20   20  14.2          Clear\n21   21  13.7  Partly cloudy\n22   22  13.4  Partly cloudy\n23   23  13.2  Partly cloudy'

In [159]:
query

'Medellin'

In [160]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-20,00,Clear,12.4,0,0
1,2023-06-20,01,Clear,12.1,0,0
2,2023-06-20,02,Partly cloudy,12.0,0,0
3,2023-06-20,03,Partly cloudy,11.8,0,0
4,2023-06-20,04,Partly cloudy,11.7,0,0
5,2023-06-20,05,Partly cloudy,11.6,0,0
6,2023-06-20,06,Partly cloudy,11.5,0,0
7,2023-06-20,07,Sunny,14.9,0,0
8,2023-06-20,08,Sunny,18.8,0,0
9,2023-06-20,09,Sunny,21.2,0,0
